# Removing Duplicative and Sparse Data

The most basic task in data cleaning is detecting and removing erroneous data. This includes duplicative data and missing or unreliable data. This is not the most glamorous task but it is enormously important. As the old adage goes, "garbage in, garbage out." Being able to wrangle and clean messy datasets is absolutely paramount to be successful, and can set you apart from others in the data science/engineering field. 

To get started, create this dataframe of weather data. 

In [ ]:
import pandas as pd

df = pd.DataFrame({
    "record_id" : ['DCMXP87EDE', 'DCMXP87EDE', 'ZMIFM3HX9G', 'HIVVXBAPS2', 'U1AA66UDES', 'B20KL5PW3L', 'FIZLY34KSQ'],
    "rain_inches" : [1.1, 1.1, 0.0, 0.0, 2.4, 11.2, 3.2],
    "tornado" : [0,0,1,0,0,0,0],
    "lightning" :[0,0,1,1,1,0,0],
    "wind_speed_mph" : [3.1, 3.1, 143.0, None, 8.1, 5.0, None],
    "severity" : ['CLEAR', 'CLEAR', 'SEVERE', 'MINOR', 'MINOR', 'MAJOR', None],
    "transmit_ind" :[1,1,1,1,1,1,1]
})

df

## Where Did the Data Come From? 

You may be tempted to dive right into writing Python code and wrangling datasets in Pandas dataframes, but let's step back for a brief moment and ask some questions. Where did this data come from? How was it collected? What sensors or data entry methods were used to collect it? Could the data be biased in any way or missing important variables? 

It is just as important, if not more so, to ask not just what the data says but also ask where it came from. This could reveal larger issues that are dirtying your data but are not detectable just by looking at the dataset alone. The data could be biased, or missing relevant data or variables for the problem being solved. If you have data that is full empty values (which we will discuss techniques for removing), you should fully understand why they are empty and whether there is a deeper problem in the process producing the data. For example, if a broken temperature sensor is recording `NA` or `NaN` values at a specific weather station, you should address fixing that sensor rather than just removing those records entirely. If a station is producing duplicate records, the software bug should be fixed rather than removing the duplicates.

There are some things you cannot quantify or apply a Pandas function to fix, and you must apply qualitative judgment to ask the right questions and address problems at the source. Once you have exhausted those questions and fully understand your dataset, then you can proceed accordingly. 

## Removing Duplicate Rows 

Let's print our dataframe of weather data. 

In [ ]:
df

Notice above how we have some questionable  data, including the top two rows being duplicates and some missing `NaN` and `None` values. Let's focus on duplicates first. 

To get all the duplicates but the first instance of a row, use the `duplicated()` function. 

In [ ]:
df.duplicated()

You can flag all instances (including the first found instance) by setting `keep=False`.

In [ ]:
df.duplicated(keep=False)

If you want to find duplicates just based on one or more columns as the key, use the `subset()` function. Below we find duplicat records using only the `record_id` field. 

In [ ]:
df.duplicated(subset=['record_id'])

We could composite our condition with multiple fields if we wished, such as `record_id` and `rain_inches`. 

In [ ]:
df.duplicated(subset=['record_id','rain_inches'])

We could use the boolean `Series` returned in the examples above to extract only those rows into a new dataframe. However, we can also use the `drop_duplicates()` function to do this as well. It accepts the same arugments as `duplicatated()` and has an `inplace` parameter for replacing the existing dataframe. 

In [ ]:
df.drop_duplicates(inplace=True)

And of course, you can always drop based on a subset. 

In [ ]:
df.drop_duplicates(subset=['record_id'], inplace=True)
df

> Note there are nearly identical functions to handle duplicates for [Index](https://pandas.pydata.org/docs/reference/api/pandas.Index.duplicated.html#pandas.Index.duplicated) and [Series](https://pandas.pydata.org/docs/reference/api/pandas.Series.duplicated.html#pandas.Series.duplicated), also called `duplicated()` and `drop_duplicates()`. They operate much in the same way as the dataframe counterpart for these functions. 

## Remove Columns with One Value

Columns that have a single value are probably not going to be useful at all for machine learning and other analysis. Therefore they are candidate for removal as long as this is not an error. Notice how the `transmit_ind` is all 1's and this is not helpful. 

In [ ]:
df

We can use the `nunique()` function to identify the number of unique values in each column as a series.

In [ ]:
df.nunique()

We can iterate the series above and track which column indices to delete, based on whether they only have one unique value. 

In [ ]:
# identify single-value columns to delete
delete_cols = [c for c,v in zip(df.columns, df.nunique()) if v == 1]
delete_cols

Finally, we can remove those columns (there will only be on in this case) by passing them to the drop function. Make sure to specify we are dropping columns by specifying `axis=1`. 

In [ ]:
df.drop(delete_cols, axis=1, inplace=True)

## Remove Columns with Few Values

When dealing with categorical values, it should be unsurprising that there are few values. In our weather data, we only expect a `True` or `False` for boolean fields. We only expect 4 or so possible values for the `weather_severity` such as `MAJOR`, `MINOR`, `CLEAR`, and `SEVERE`. Rarely we should consider discrete variables like this to be too sparse to use. 

However, dealing with nuermical/continous values (decimals) is a different story. When a numerical column has few values, there may not be much variance to make meaningful model predictions. If this is indeed the case, they should be removed. This is not always the case so be sure to remove when it is absolutely certain they do not add value. Sometimes the choice of model will impact this decision as well, as linear models often depend on some variance for a meaningful distribution of the data.

Let's bring in a different dataset for this example: a wine quality dataset with different wine attributes. 

In [ ]:
wine_df = pd.read_csv('https://raw.githubusercontent.com/thomasnield/machine-learning-demo-data/master/regression/winequality-red.csv')
wine_df

One metric that might guide us to columns with low numbers of unique values is, for each column, the proportion of unique values out of all rows. Below we take each column, and divide the number of unique values by the number of rows. 

In [ ]:
n_rows, n_cols = wine_df.shape

for i in range(n_cols):
    unique_num = wine_df.iloc[:, i].nunique()
    percentage = float(unique_num) / n_rows * 100 
    print(f'{i}, {unique_num}, {round(percentage,2)}%')

As you can see above, there are some columns with very low percentages of unique values. The categorical ones are to be expected, like the last column `quantity`. But some columns like `alcohol` (at position 11) and `free_sulfur_dioxide` (at position 5) are really low. 

Let's say we wanted to remove columns with 5% or less unique values. Let's adapt our `for` loop above to extract column labels that have a percentage of unique values of `.05` or less. 

In [ ]:
delete_cols = []

n_rows, n_cols = wine_df.shape

for i in range(n_cols):
    unique_num = wine_df.iloc[:, i].nunique()
    percentage = float(unique_num) / n_rows  
    if percentage <= .05:
        delete_cols.append(wine_df.columns[i])
    
delete_cols

We will then take those three columns and then drop them. You will then notice those columns are removed. 

In [ ]:
wine_df.drop(delete_cols, axis=1, inplace=True)
wine_df

## Remove Columns with Low Variance

Another way to approach this problem of columns with few unique values is to calculate the variance and use that as a cutoff threshold. Recall that variance $ \sigma^2 $ is a measure in statistics that averages the squared differences between each observed value $ x_i $ and the mean $ \mu $ of those values. In other words, to calculate variance  square the difference between each data point $ x_i $ and the mean $ \mu $, sum them, and divide by number of elements $ n $. 

$$
\sigma^2 = \frac{\sum_{i=1}^{n}(x_i - \mu)^2} {n}
$$ 

Let's load our wine dataset again to start over and bring those removed columns back. 

In [ ]:
wine_df = pd.read_csv('https://raw.githubusercontent.com/thomasnield/machine-learning-demo-data/master/regression/winequality-red.csv')
wine_df

The lower the variance, the less unique values we can expect. There is a helpful utility `VarianceThreshold` in scikit-learn that can be used to remove features based on variance. Typicaly, we want more variance for modeling purposes in statistics and machine learning. Having too little variance in a feature is not going to be useful. Let's declare an instance of `VarianceThreshold` here and set its threshold to `.01`. The higher this parameter is, the more columns it will eliminate due to higher cutoffs for variance. 

In [ ]:
from sklearn.feature_selection import VarianceThreshold

vt = VarianceThreshold(threshold=.05)

Next let's extract just the input variable columns by selecting all but the last column (which is `quality`). Then we pass it to the `VariableThreshold`'s `fit_transform()` function to get the columns of data that met that threshold. 

In [ ]:
X = wine_df.iloc[:,:-1]
X_threshold = vt.fit_transform(X)
X_threshold

So how many columns made it through and met that variance threshold? Let's take a look at the shape and count the number of columns before and after the transformation.

In [ ]:
print(f"NUM FEATURES BEFORE: {X.shape[1]}")
print(f"NUM FEATURES AFTER: {X_threshold.shape[1]}")

So two columns were eliminated. Unfortunately, in this transformation our `DataFrame` was turned into a NumPy `ndarray`. Thankfully, there is a `get_support()` function on the `VarianceThreshold` to return the indices of the columns that pass the cutoff. We can then pass that back to the `columns` property to get the column indices, and then use that to select those columns off our dataframe. 

In [ ]:
wine_df[wine_df.columns[vt.get_support(indices=True)]]

As you can see, all but 5 of those columns have been eliminated and did not pass the variance threshold. 

## Exercise

Below is a dataframe of thermostate data. Complete the code by replacing question marks "?" to remove duplicative records and any columns with 3 or less unique values. 

In [ ]:
import pandas as pd

df = pd.DataFrame({
    "record_id" : ['OVUTJE','OVUTJE','WI4QEX','WI4QEX','FS40NF','O64LIT','U888EA'],
    "temperature" : [65.2, 65.2, 47.2, 57.2, 57.4, 23.4, 27.5], 
    "humidity" : [.8, .7, .7, .6, .7, .7, .8],
    "stable" : [True, True, True, True, True, True, True]
})

# drop duplicates
df.?

# remove columns with 3 or less unique values
delete_cols = []

n_rows, n_cols = df.shape

for i in range(n_cols):
    unique_num = df.iloc[:, i].nunique()
    if unique_num <= 3:
        delete_cols.append(df.columns[i])
    
df.?(delete_cols, axis=?, inplace=True)
df

### SCROLL DOWN FOR ANSWER
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
v 

In [ ]:
import pandas as pd

df = pd.DataFrame({
    "record_id" : ['OVUTJE','OVUTJE','WI4QEX','WI4QEX','FS40NF','O64LIT','U888EA'],
    "temperature" : [65.2, 65.2, 47.2, 57.2, 57.4, 23.4, 27.5], 
    "humidity" : [.8, .7, .7, .6, .7, .7, .8],
    "stable" : [True, True, True, True, True, True, True]
})

# drop duplicates
df.drop_duplicates(inplace=True)

# remove columns with 3 or less unique values
delete_cols = []

n_rows, n_cols = df.shape

for i in range(n_cols):
    unique_num = df.iloc[:, i].nunique()
    if unique_num <= 3:
        delete_cols.append(df.columns[i])
    
df.drop(delete_cols, axis=1, inplace=True)
df